<a href="https://colab.research.google.com/github/devorahst/Test/blob/main/ChiSquareExample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Chi-Square Test of Independence**
In this walkthrough, we will go over an example of using Chi-Square Test of Independence to establish association between two of the features from our SAK datset. We will be using the feature 'DNAKitUsed' and our dependent variable, 'CODISNDISeligibleProfile'. 

# Part 1. Set-Up
First, we must download the dataset. Upon running the cell, you will be prompted to login to your Gmail account. You will then be provided with a one-time use code to copy and paste into the slot below. After hitting enter, the dataset will load into this script.

*See [IO Notebook](https://colab.research.google.com/drive/1fuF8iahEqBFV62Y6OoiEViUqHo-DbXrT) for more information about set-up and interacting with our dataset

In [ ]:
#pulls up our SAK dataset
#@title Upload Dataset
file_id = "13DLmmbYXonl9alHR4VobfeTuA_IxlQxZ" #@param {type:"string"}
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import auth
auth.authenticate_user()

from googleapiclient.discovery import build
drive_service = build('drive', 'v3')

import io
from googleapiclient.http import MediaIoBaseDownload

request = drive_service.files().get_media(fileId=file_id)
downloaded = io.BytesIO()
downloader = MediaIoBaseDownload(downloaded, request)
done = False
while done is False:
  _, done = downloader.next_chunk()

fileId = drive.CreateFile({'id': file_id }) #DRIVE_FILE_ID is file id example: 1iytA1n2z4go3uVCwE_vIKouTKyIDjEq
print(fileId['title'])  
fileId.GetContentFile(fileId['title'])  # Save Drive file as a local file

!pip install -U scikit-learn

from scipy.stats import chi2_contingency
from scipy import stats
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})

with open(fileId['title'], encoding="utf8", errors='ignore') as f:
  df = pd.read_csv(f)

df = df.apply(pd.to_numeric, errors='ignore')
predictedVariable = "CODISNDISeligibleProfile"
df = df.replace(r'^\s+$', np.nan, regex=True)
df = df.replace({np.nan: "No Response"})
df = df.applymap(str)

#delete rows that don't have a value for CODISNDISeligibleProfile
df = df[df[predictedVariable] != "No Response"]

predictedFeatures = ['CODISNDISeligibleProfile', 'SDISeligibleprofile']  

numericalFeatures = ['Age', 'Timebetweenassaultandexaminhours', 'PainLevel', 'MulitipleSuspectNumber', 
                     'NumberofUnknownresponses', 'NumberAssaultiveActs', 'Numberofphysicalinjuries', 'Numberofgentialinjuries',
                     'NumberOFitemsTested', 'TimeBetweenCollectAndDNAext', 'TimeBetweenSubmissionANDtesting', 'NumberOfswabsQuantMaleDNA',
                     'NumberOfswabsDNAanalysis', 'NumberofSTRDNAloci', 'NumberOFswabsSTRDNAprofile', 'NumberOfYSTRDNAloci']

categoricalFeatures = ['Site', 'EXAMbySANE', 'YearKitCollected', 'KITbroughtTOcrimelab', 'KITlengthofSubmissionTime',
                       'UnderAge18', 'Gender', 'ExamDeclined', 'Noninterview', 'Race', 'PriorHxofSAover14',
                       'PriorHxofSAunder14', 'Student', 'Military', 'Pain', 'PainLocation1','PainLocation2', 
                       'PainLocation3', 'PainLocation4','PainTreatment', 'PermanentAddress', 'CurrentPhysicalmedprob',
                       'MedProbChronic', 'MedProbInfection', 'MedProbBlood', 'MedProbCardiac', 'MedProbEar', 'MedProbEndocrine',
                       'MedProbEye', 'MedProbGI', 'MedProbGU', 'MedProbGYN', 'MedProbImmune', 'MedProbMusculoskeletal', 'MedProbNeurological',
                       'MedProbOral', 'MedProbRenal', 'MedProbRespiratory', 'MedProbSkin', 'MedProbOther', 'Medication',
                       'PsychotropicMEDuse', 'PsychotropicANTIPSYCHOTICSatypical', 'PsychotropicSTIMULANTuse', 'PsychotropicANTIANXIETY', 
                       'PsychotropicANTIDEPRESSANTS', 'PsychotropicANTISEIZUREbipolar', 'PsychotropicADDICTIONmeds','PsychotropicSLEEPaid', 'PsychotropicOTHER', 
                       'PsychotropicANTIPSYCHOTICStypical', 'PolypharmacyPsychMeds', 'ImmunizationstatusTETANUS', 'ReceivedTetanus',
                       'ImmunizationstatusHEP', 'ReceivedHepB', 'Sexualcontactwithin120hours', 'Selfdisclosurementalillness', 'MIdepression',
                       'MIanxiety', 'MIPTSD', 'MIpsychoticDisorders', 'MIadhd', 'MIpersonalitydisorder', 'MIbipolar', 'MIeatingdisorder', 'MIdrugalcoholdisorders', 
                       'MIother', 'SelfDiscolsureMentalillnessORuseofpsychotropics', 'OnlineMeetingOFsuspect', 'Suspectrelationship',
                       'Locationofassault', 'PatientActionScratch', 'PatientActionBite', 'PatientActionHit', 'PatientActionKick', 'PatientActionOther',
                       'Suspectrace', 'SuspectactionVERBAL', 'SuspectactionsGRABBEDHELD', 'SuspectactionsPHYSICALBLOWS', 'SuspectactionsSTRANGLEDCHOKED',
                       'SuspectactionsWEAPON', 'SuspectactionsRESTRAINTS', 'SuspectactionsBURNED', 'MultipleSuspects', 'SuspectedDrugfacilitated',
                       'Patientdruguse', 'PatientETOHuse', 'Suspectdruguse', 'SuspectETOHuse', 'PatientSuspectETOHordrug', 'LossOFconsciousnessORawareness',
                       'OneORmoreunknownanswer', 'Unknownanswerto4ormorequestions', 'UnknownanswertoALL', 'AsleepANDawakenedtoassault', 'MemoryLoss',
                       'LossOfconsciousness', 'DecreasedAwareness', 'TonicImmobility', 'Detachment', 'NOSApatientsVAGINApenis', 'NOSApatientsVAGINAfingerhand',
                       'NOSApatientsVAGINAmouth', 'NOSApatientsVAGINAobject', 'NOSApatientsANUSpenis', 'NOSApatientsANUSfingerhand', 'NOSApatientsANUSmouth', 
                       'NOSApatientsANUSobject', 'NOSApatientsPENISgenitals', 'NOSApatientsPENISfinger', 'NOSApatientsPENISmouth', 'NOSApatientsPENISobject', 
                       'NOSApatientsMOUTHpenis', 'NOSApatientsMOUTHfinger', 'NOSApatientsMOUTHmouth', 'NOSApatientsMOUTHobject', 'SUSPECTmouthcontactGENITALS', 
                       'SUSPECTmouthcontactMOUTH', 'SUSPECTmouthcontactOTHER', 'SUSPECTmouthcontactOTHERsite', 'HANDSofSuspectBreast', 'HANDSofSuspectExtremities', 
                       'HANDSofSuspectOther', 'Ejaculation', 'CONDOMuse', 'LUBRICATIONuse', 'SuspectWASHEDpatient', 'SuspectINJUREDbypatient', 'PostassaultURINATED', 
                       'PostassaultDEFECATED', 'PostassaultDOUCHED', 'PostassaultVOMITED', 'PostassaultGARGLED', 'PostassaultBRUSHEDTEETH', 'PostassaultATEdrank', 
                       'PostassaultBATHED', 'PostassaultGENITALWIPE', 'PostassaultCHANGEDCLOTHING', 'PostassaultREMOVEDInserted', 'PhysicalORmentalimpairment', 'Physicalinjury', 
                       'LPIhead', 'LPIneck', 'LPIbreasts', 'LPIchestback', 'LPIabdomen', 'LPIextremities', 'TPIlaceration', 'TPIecchymosis', 'TPIabrasion', 'TPIredness', 
                       'TPIswelling', 'TPIbruise', 'TPIpetechiae', 'TPIincision', 'TPIavulsion', 'TPIdiscoloredmark', 'TPIpuncturewound', 'TPIfracture', 
                       'TPIbitemark', 'TPIburn', 'TPImissingorbrokenTEETH', 'TPIconjunctivalhemorrhage', 'Genitalinjury', 'LGIinnerthighs', 'LGIclitoralhoodclitoris', 
                       'LGIlabiamajora', 'LGIlabiaminora', 'LGIperiurethraltissueURETHRA', 'LGIperihymenaltissue', 'LGIhymen', 'LGIvagina', 'LGIcervix', 'LGIfossanavicularis', 
                       'LGIposteriorfourchette', 'LGIperineum', 'LGIperineum', 'LGIanalrectal', 'LGIbuttocks', 'LGImalePerianalperineum', 'LGIglanspenis', 'LGIpenileshaft', 
                       'LGImaleURETHRALmeatus', 'LGIscrotum', 'LGItestes', 'LGImaleanus', 'LGImalerectum', 'TGIlaceration', 'TGIecchymosis', 'TGIabrasion', 'TGIredness', 
                       'TGIswelling', 'TGIbruise', 'TGIpetechiae', 'TGIincision', 'TGIavulsion', 'TGIdiscoloredmark', 'TGIpuncturewound', 'ToludineDYEuptake', 'HIVnPEP', 
                       'UQuikcollected', 'Yscreen', 'NumberItemsWITH3cutoff', 'ItemsAnalyzed1', 'ItemsAnalyzed2', 'ItemsAnalyzed3', 'ItemsAnalyzed4', 'ItemsAnalyzed5', 
                       'ItemsAnalyzed6', 'ItemsAnalyzed7', 'ItemsAnalyzed8', 'ItemsAnalyzed9', 'ItemsAnalyzed10', 'TypesOFitemsTested', 'RandomSample20142015', 
                       'YearofDNAextraction', 'LocationOfTesting','DANYfundedSAK', 'DNAKitUsed', 'SerologyDoneBeforeDNA', 'QuantMaleDNAFound', 'QuantMaleSwabLoc1', 
                       'QuantMaleSwabLoc2', 'QuantMaleSwabLoc3', 'QuantMaleSwabLoc4', 'QuantMaleSwabLoc5', 'Swab1ToDNAanalysis', 'Swab2ToDNAanalysis', 
                       'Swab3ToDNAanalysis', 'Swab4ToDNAanalysis', 'ProbableSTRDNAprofileOFsuspect', 'ProfileofSTRDNAloci', 'ProbableYSTRDNAprofile', 'ProfileOfYSTRDNAloci', 
                       'SwabLocationYSTRDNA', 'SecondSwabLocationYSTRDNA', 'SwabFromSuspectwithVictimDNA', 'ExcludeSuspect', 'ConsensualPartnerStandardSubmitted', 
                       'STRDNAProbableprofileTYPE', 'CODISprofileHit', 'STRDNAkitUsed', 'SUSPECTmouthcontactBREASTS', 'Swab1LocationSTRDNAprofile', 'Swab2LocationSTRDNAprofile',
                       'Swab3LocationSTRDNAprofile', 'SuspectStandardSubmitted', 'CODISNDISreasons', 'CODISSDISreasons']

#unusedFeatures and stringFeatures are columns that contain data that was relevant to medical professionals and for legal purposes, 
#but that aren't useful for our feature association or for predicting eligibility
unusedFeatures = ['filter_$', 'PainTreatmentYesNo', 'GenderMaleFemale', 'DVsuspect', 'RacePrimaryGroups', 'IPSAcombined', 'STRDNAcompleted', 
                  'PhysicalInjuryNOunknown', 'GenitalInjuryNOunknown']

stringFeatures = ['DeIdentifiedCase', 'Raceother', 'SchoolName', 'MilitaryBranchName', 'AddressIfnotPermanent', 'Currentmedprobtext',
                  'MedProbOtherText', 'Medicationtext', 'Sexualcontactwithin120hoursTYPE', 'SelfdisclosureMItype', 'OnlineMeetingName', 'SuspectrelationshipOTHER',
                  'LocationofassaultOTHER', 'Surfaceofassault', 'PatientActionOtherTEXT', 'SuspectraceOTHER', 'SuspectOTHERactions', 'NOSApatientsVAGINAobjectdescription',
                  'NOSApatientsANUSobjectdescription', 'NOSApatientsPENISobjectdescription', 'NOSApatientsMOUTHobjectdescription', 'EjaculationSITE', 'LUBRICATIONtype',
                  'SuspectINJUREDbypatientexplanation', 'Impairmentdescription', 'UBFSnumber', 'ISPnumber', 'DateSubmittedUBFS', 'DateofDNAextractionReport',
                  'BodySwabLocQuant', 'BodySwabDNAanalysis', 'BodySwabLocationSTRDNA', 'BodySwabYSTRDNA', 'ISPnotes2020', 'UBFSnotes2020', 'UBFSnotes2018', 'UBFSnotes2014']


MasterValentine_UpdatedCODIS_Feb12_2021.csv


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,1,2,3,4,5,9,301) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#Part 2. Chi-Square
Now we can run our chi-square test. We do this by first finding our bonferroni correction for our p-value cutoff of significance, then create our contingency table, and finally use our table in a chi-square model. 

1. Explanation of the Bonferroni Correction
2. Contingency Table 
3. Chi Square Test and Walkthrough of our Findings

###Bonferroni Correction
If we only had one hypothesis to test (i.e. the association of one feature to our predicted variable), we could safely use an alpha value of .05. But because we are comparing 314 separate features, the risk of a type 1 error (falsely rejecting our null hypothesis of two features having no significant association) occurring is increased. We can correct for this by using the Bonferroni Correction. We calculate this by taking our desired alpha level of .05 and dividing it by the total number of hypotheses used (or features we are comparing). In this example, we will use .05/314 to receive a p-value cutoff of 1.592357e-04. That means that any p-values we calculate above this cutoff are not significant and we will fail to reject our null hypothesis that the given feature is not associated with an eligible profile. 

In [ ]:
#@title Bonferroni Cutoff
#Bonferroni cutoff of significance 
scientific_notation = "{:e}".format(12300000)

print("The Bonferroni cutoff is " + "{:e}".format(.05/len(df.columns)) + ". Features with p-values above this are not significant.")
#Number of features considered
print("There are", str(len(df.columns)), "features being considered.")

The Bonferroni cutoff is 1.592357e-04. Features with p-values above this are not significant.
There are 314 features being considered.


### Example Contingency Table
 A contingency table is an arrangement that shows the relation between two features. Rows of the contingency table show the different categories for one of the features while the categories for the other feature are presented by the columns. Cell values show the count of total cases for each combination of categories. In this case, we are comparing the type of DNAKitUsed to obtaining a CODISNDISeligibleProfile. Below are the keys taken from the [code book](https://docs.google.com/document/d/18PDTuK0lshc193lXA3b7UDgcRzMoEEGA/edit?rtpof=true) for the features DNAKitUsed and CODISNDISeligibleProfile. In this example, we will try to find if the type of DNA kit used is strongly associated with an eligible profile result.



**Key for DNAKitUsed**
*   1 = ID+
*   2 = Yfiler
*   3 = Both ID+ and Yfiler
*   4 = Globalfiler
*   5 = Globalfiler and Yfiler
*   6 = None
*   7 = Promega STR
*   8 = Promega YSTR
*   9 = Promega STR and YSTR
*   No Response = Data not collected in exam or not reported

**Numeric Key for CODISNDISeligibleProfile**
*   0 = No (Not an eligible profile)
*   1 = Yes, uploaded (Is an eligible profile)

In [ ]:
contigency= pd.crosstab(df['DNAKitUsed'], df[predictedVariable]) 
# contigency = contigency.drop(index='2')
# contigency = contigency.drop(index='3')
# contigency = contigency.drop(index='5')
# contigency = contigency.drop(index='6')
# contigency = contigency.drop(index='9')
# contigency = contigency.drop(index='No Response')

print(contigency)


CODISNDISeligibleProfile     0     1
DNAKitUsed                          
1                          134   229
2                          159    17
3                          202   254
4                         1194  1027
5                           19    18
6                         1392     3
7                          246   299
9                            1     0
No Response                449     0


Using the contingency table, it is easier to see big differences between eligible and noneligible profiles for each kit type. We aren't as interested in results for row 9 due to small sample size or the "No Response" row, but a kit coded as 2 or 6 stands out as not producing many eligible CODISNDIS profiles. If we take a look at what these two rows actually code for, we see that a 2 represents the use of a YFiler kit and 6 codes for no kit being used. It makes sense that if no kit were used, we wouldn't expect to get a profile hit. However, we may want to look more into the use of a YFiler kit to see why this kit doesn't seem to produce many eligible profiles.

Overall, any of these factors could influence whether our feature of DNAKitUsed returns a significant p-value when we run our Chi-Square Test. We should keep this in mind as we continue our analysis of our dataset (And maybe run Pairwise testing to look into how each kit type affects our p-value). 

###Example of Chi-Square and Explanation of Our Findings 

Using the scipy.stats.chi2_contingency function, we can easily find the p-value assigned to the association between our two features. This function also returns the test statistic used, the degrees of freedom necessary to calculate our result, and the expected frequencies based on the marginal sums of our table. For our purposes, we really only care about the p-value that is returned from this function. If our p-value is less than our bonferroni correction value of 1.592357e-04, we reject our null hypothesis that DNAKitUsed and CODISNDISeligibleProfile are not associated. 

In [ ]:
#c - the test statistic
#p - the probability
#dof - degrees of freedom
#expected - The expected frequencies, based on the marginal sums of the table.

cutoff = 0.05 / len(df.columns)
c, p, dof, expected = chi2_contingency(contigency) 
if (p < cutoff):
  print(str(p) + " is less than 1.592357e-04." )

3.486807150240603e-258 is less than 1.592357e-04.


The P-Value we obtained from our Chi-Square was less than our cutoff value, so we reject our null hypothesis. That means that we can assume that the feature DNAKitUsed is associated with the feature CODISNDISeligibleProfile. As noted before, we would probably want to take this feature and dig deeper with Pairwise testing to note which categories have the strongest pull towards obtaining this significant p-value. 